## Daten Laden

In [1]:
file_path = "BIG_2025-05-09__22_03_20-normalized-supplemented.csv"
print(file_path)

BIG_2025-05-09__22_03_20-normalized-supplemented.csv


In [2]:
import pandas as pd

df = pd.read_csv(file_path)

# Drop command column
df = df.drop(columns=['command'])

# Display the first few rows to understand the structure
df.head()

,watt-median,L1-dcache-loads:u,dTLB-stores:u,dTLB-stores:u-square,dTLB-store-misses:u,dTLB-store-misses:u-square,iTLB-load-misses:u,iTLB-load-misses:u-square,branch-loads:u,branch-loads:u-square,...,branch-instructions:u-square,branch-misses:u,cpu-cycles:u,instructions:u,instructions:u-square,mem-stores:u,mem-stores:u-square,ref-cycles:u,slots:u,topdown-br-mispredict:u
0,0.000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,11.415,1.902017e+09,7.665781e+08,5.876420e+17,211483.675442,4.472534e+10,109381.308269,1.196427e+10,2.430663e+09,5.908124e+18,...,5.676251e+18,1.594215e+07,3.098193e+09,1.100679e+10,1.211494e+20,9.093660e+08,8.269465e+17,3.134993e+09,1.685133e+10,2.316390e+09
2,9.810,9.993717e+08,4.515674e+08,2.039131e+17,445.960164,1.988805e+05,33895.824578,1.148927e+09,8.314625e+08,6.913299e+17,...,6.956822e+17,1.214509e+07,3.456444e+09,7.607542e+09,5.787469e+19,4.524578e+08,2.047180e+17,3.468530e+09,2.071757e+10,1.787447e+09
3,9.535,1.340750e+09,1.340901e+09,1.798016e+18,64.698175,4.185854e+03,88.915702,7.906002e+03,4.268624e+09,1.822115e+19,...,1.424965e+19,6.593654e+06,2.874016e+09,1.318918e+10,1.739544e+20,1.341064e+09,1.798452e+18,3.465344e+09,2.080430e+10,6.528354e+08
4,8.590,4.624207e+08,4.613487e+08,2.128427e+17,64.146327,4.114751e+03,277.392659,7.694669e+04,9.248547e+08,8.553562e+17,...,8.501592e+17,1.230838e+04,2.872220e+09,1.055935e+10,1.114998e+20,5.558364e+08,3.089541e+17,3.463458e+09,1.724837e+10,2.155015e+05


## Korrelationsmatrix

In [3]:
corr = df.corr()
corr.style.background_gradient(cmap='RdYlGn')

,watt-median,L1-dcache-loads:u,dTLB-stores:u,dTLB-stores:u-square,dTLB-store-misses:u,dTLB-store-misses:u-square,iTLB-load-misses:u,iTLB-load-misses:u-square,branch-loads:u,branch-loads:u-square,branch-load-misses:u,L1-dcache-load-misses:u,L1-dcache-load-misses:u-square,branch-instructions:u,branch-instructions:u-square,branch-misses:u,cpu-cycles:u,instructions:u,instructions:u-square,mem-stores:u,mem-stores:u-square,ref-cycles:u,slots:u,topdown-br-mispredict:u
watt-median,1.000000,0.634167,0.425705,0.255651,0.204448,0.203466,0.297090,0.234437,0.436250,0.263891,0.182733,0.219228,0.189801,0.443515,0.281646,0.182874,0.700990,0.535222,0.440029,0.422638,0.253487,0.723473,0.763070,0.210304
L1-dcache-loads:u,0.634167,1.000000,0.730219,0.512393,0.153083,0.152259,0.282905,0.201918,0.442518,0.264074,0.111818,0.182598,0.148648,0.459767,0.300726,0.112225,0.145363,0.419689,0.384888,0.723085,0.500175,0.115273,0.287823,0.145257
dTLB-stores:u,0.425705,0.730219,1.000000,0.911424,0.061830,0.061424,0.121142,0.081625,0.526149,0.411779,0.068818,-0.069497,-0.091614,0.554274,0.467148,0.070996,0.099445,0.452811,0.425095,0.994134,0.883727,0.094056,0.194055,0.067608
dTLB-stores:u-square,0.255651,0.512393,0.911424,1.000000,0.001632,0.001396,0.005246,-0.006515,0.434000,0.355290,-0.012987,-0.035111,-0.050967,0.470719,0.421132,-0.011114,0.093345,0.391440,0.385758,0.918031,0.981488,0.051549,0.101579,-0.035732
dTLB-store-misses:u,0.204448,0.153083,0.061830,0.001632,1.000000,0.999997,0.656734,0.764152,0.198923,0.167103,0.115319,-0.010123,-0.014904,0.192022,0.162962,0.114412,0.029144,0.101850,0.079955,0.084502,0.016020,-0.079487,-0.045210,0.137098
dTLB-store-misses:u-square,0.203466,0.152259,0.061424,0.001396,0.999997,1.000000,0.656046,0.763832,0.198665,0.167037,0.115199,-0.010918,-0.015505,0.191766,0.162896,0.114286,0.028359,0.101387,0.079530,0.084130,0.015836,-0.079860,-0.045963,0.136966
iTLB-load-misses:u,0.297090,0.282905,0.121142,0.005246,0.656734,0.656046,1.000000,0.952847,0.162523,0.081259,0.107751,-0.016857,-0.038763,0.156635,0.077002,0.110669,0.085773,0.108554,0.078263,0.121305,0.003413,-0.020184,-0.008635,0.178734
iTLB-load-misses:u-square,0.234437,0.201918,0.081625,-0.006515,0.764152,0.763832,0.952847,1.000000,0.173116,0.111659,0.125196,-0.018456,-0.027540,0.167085,0.107585,0.127279,0.038713,0.086167,0.053026,0.091377,-0.000074,-0.044814,-0.042008,0.184563
branch-loads:u,0.436250,0.442518,0.526149,0.434000,0.198923,0.198665,0.162523,0.173116,1.000000,0.924076,0.168297,0.004409,-0.007514,0.986834,0.918766,0.148220,0.097752,0.684407,0.654953,0.523248,0.418641,0.117225,0.153454,0.161177
branch-loads:u-square,0.263891,0.264074,0.411779,0.355290,0.167103,0.167037,0.081259,0.111659,0.924076,1.000000,0.107807,-0.032086,-0.037063,0.895214,0.960984,0.087779,0.027577,0.558557,0.560951,0.409824,0.341783,0.055894,0.108332,0.084771


## 1. Versuch (Regression)

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, max_error

# --------------------------------------------
# Kein Intercept-Modell: fit_intercept=False setzt den Intercept auf 0.
# --------------------------------------------

# Hyperparameter für Balance: Einflussfaktor und Exponent für Gewichtung der Fehler
alpha = 20.0       # Betonung der Ausreißer
exponent = 1.5     # leicht überlineare Gewichtung

# Zielvariable: erste Spalte enthält die zu schätzende Zahl
y = df.iloc[:, 0]
# Potenzielle Features: alle anderen Spalten
potential_features = df.drop(columns=[df.columns[0]])

# Auswahl der 6 besten Features basierend auf f_regression
selector = SelectKBest(score_func=f_regression, k=6)
X_selected = selector.fit_transform(potential_features, y)
selected_feature_names = potential_features.columns[selector.get_support()]
X = pd.DataFrame(X_selected, columns=selected_feature_names, index=df.index)

# Aufteilen in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training des ungewichteten Modells ohne Intercept
base_model = LinearRegression(fit_intercept=False)
base_model.fit(X_train, y_train)

# Berechnung der Trainingsfehler für Gewichtung
def train_and_compute_errors(model, X, y):
    preds = model.predict(X)
    return np.abs(y - preds)

train_errors = train_and_compute_errors(base_model, X_train, y_train)
# Normieren und Nichtlinear transformieren
errors_norm = train_errors / train_errors.max()
sample_weights = 1 + alpha * np.power(errors_norm, exponent)

# Training des gewichteten Modells ohne Intercept
model_balanced = LinearRegression(fit_intercept=False)
model_balanced.fit(X_train, y_train, sample_weight=sample_weights)

# Evaluation: ungewichtet vs. gewichtet
def evaluate(model, X, y):
    preds = model.predict(X)
    return {
        'MSE': mean_squared_error(y, preds),
        'Max Error': max_error(y, preds)
    }

results = []
for name, model in [('Ungewichtet', base_model), ('Gewichtet', model_balanced)]:
    metrics = evaluate(model, X_test, y_test)
    results.append({'Modell': name, **metrics})

result_df = pd.DataFrame(results)
print(result_df)

# Ausgabe der ausgewählten Features
print("Ausgewählte Features:", selected_feature_names.tolist())

# Ausgabe der Koeffizienten (kein Intercept)
print("\n--- Ungewichtetes Modell (kein Intercept) ---")
print("Koeffizienten:")
for feat, coef in zip(selected_feature_names, base_model.coef_):
    print(f"  {feat}: {coef}")

print("\n--- Gewichtetes Modell (kein Intercept) ---")
print("Koeffizienten:")
for feat, coef in zip(selected_feature_names, model_balanced.coef_):
    print(f"  {feat}: {coef}")


        Modell       MSE  Max Error
0  Ungewichtet  0.115380   0.719894
1    Gewichtet  0.112352   0.773672
Ausgewählte Features: ['L1-dcache-loads:u', 'branch-instructions:u', 'cpu-cycles:u', 'instructions:u', 'ref-cycles:u', 'slots:u']

--- Ungewichtetes Modell (kein Intercept) ---
Koeffizienten:
  L1-dcache-loads:u: 7.648005652653276e-10
  branch-instructions:u: 5.827050353287268e-11
  cpu-cycles:u: 8.426005689896472e-10
  instructions:u: 6.198081190045516e-11
  ref-cycles:u: 1.3521300939411315e-09
  slots:u: 3.457015279591285e-11

--- Gewichtetes Modell (kein Intercept) ---
Koeffizienten:
  L1-dcache-loads:u: 7.886563445880864e-10
  branch-instructions:u: 3.322996314075699e-10
  cpu-cycles:u: 1.1650503993504607e-09
  instructions:u: 3.0849775100557654e-11
  ref-cycles:u: 6.659123282528457e-10
  slots:u: 1.0850461620013055e-10
